In [2]:
#import csv
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
!aws s3 ls s3://508-team4/data/

                           PRE cpi/
                           PRE la_compensation/
                           PRE sf_compensation/
                           PRE sj_compensation/
2023-03-17 04:58:12          0 
2023-03-19 18:54:13     802519 sj_compensation_2013.csv
2023-03-19 18:54:16     905397 sj_compensation_2014.csv
2023-03-19 18:54:18     964227 sj_compensation_2015.csv
2023-03-19 18:54:19     965656 sj_compensation_2016.csv
2023-03-19 18:54:21    1055695 sj_compensation_2017.csv
2023-03-19 18:54:22    1008253 sj_compensation_2018.csv
2023-03-19 18:54:23    1054198 sj_compensation_2019.csv
2023-03-19 18:54:24    1026127 sj_compensation_2020.csv
2023-03-19 18:54:26    1033928 sj_compensation_2021.csv


In [4]:
!aws s3 ls s3://508-team4/data/la_compensation/

2023-03-26 00:55:17          0 
2023-03-26 08:01:18  147692906 la_compensation.csv


In [5]:
!aws s3 ls s3://508-team4/data/sf_compensation/

2023-03-26 00:57:08          0 
2023-03-26 04:57:06  185730616 sf_compensation.csv


In [53]:
!aws s3 ls s3://508-team4/data/sj_compensation/

2023-03-26 00:58:31          0 
2023-03-26 08:03:56    9574703 sj_compensation.csv


In [54]:
la_comp_df = pd.read_csv('s3://508-team4/data/la_compensation/la_compensation.csv')

sf_comp_df = pd.read_csv('s3://508-team4/data/sf_compensation/sf_compensation.csv')

sj_comp_df = pd.read_csv('s3://508-team4/data/sj_compensation/sj_compensation.csv')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (0,8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [57]:
sj_comp_df

,name,department,job_title,total_cash,base_salary,overtime,health,retired,year,city_id,irregular_cash,retirement,other_benefits,total_benefits,total_compensation
0,Bustillos Steven D,Police,Police Sergeant,286137.69,90888.00,89867.88,13640.50,Yes,2013,1,105381.81,74429.71,0.00,88070.21,374207.90
1,Figone Debra J,City Manager,City Manager U,248564.84,227975.02,NaN,15166.00,Yes,2013,1,20589.82,151550.08,0.00,166716.08,415280.92
2,Guerra Daniel P,Police,Police Officer,241039.12,97198.40,132104.54,15371.56,No,2013,1,11736.18,79821.51,0.00,95193.07,336232.19
3,Moore Christopher M,Police,Chief Of Police U,233540.32,15319.54,NaN,1371.81,Yes,2013,1,218220.78,11204.16,0.00,12575.97,246116.29
4,Vasquez Hector M,Police,Police Officer,230469.85,97198.40,124552.69,16345.32,No,2013,1,8718.76,79821.51,0.00,96166.83,326636.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71941,Sialehaevala Reed,Parks/Rec & Neigh Serv P R N S,Recreation Leader PT,24.56,24.56,NaN,NaN,NaN,2021,1,0.00,0.92,0.43,1.35,25.91
71942,Edgar Tigranyan,Police,Police Officer,24.52,NaN,6.40,NaN,NaN,2021,1,18.12,0.00,0.43,0.43,24.95
71943,Phylicia Sanchez,Parks/Rec & Neigh Serv P R N S,Recreation Leader PT,12.28,12.28,NaN,NaN,NaN,2021,1,0.00,0.46,0.22,0.68,12.96
71944,Justin Ruehlig,Parks/Rec & Neigh Serv P R N S,Recreation Leader PT,12.28,12.28,NaN,NaN,NaN,2021,1,0.00,0.46,0.22,0.68,12.96


In [55]:
import boto3
import sagemaker

In [56]:
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

In [7]:
#bucket = '508-team4/data'

In [8]:
#!pip install --disable-pip-version-check -q PyAthena==2.1.0
from pyathena import connect

In [15]:
database_name = "compensation"

In [17]:
bucket

'sagemaker-us-east-1-117315948243'

In [16]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = f"s3://{bucket}/athena/staging"

In [18]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [19]:
statement = f"CREATE DATABASE IF NOT EXISTS {database_name}"
print(statement)

CREATE DATABASE IF NOT EXISTS compensation


In [20]:
pd.read_sql(statement, conn)

""


In [37]:
la_comp_df.to_csv('s3://team4-data-athena-files/la_comp.csv',index=False)

In [38]:
sf_comp_df.to_csv('s3://team4-data-athena-files/sf_comp.csv',index=False)

In [39]:
sj_comp_df.to_csv('s3://team4-data-athena-files/sj_comp.csv',index=False)

In [58]:
sj_comp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71946 entries, 0 to 71945
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   name                71946 non-null  object 
 1   department          71946 non-null  object 
 2   job_title           71946 non-null  object 
 3   total_cash          71946 non-null  float64
 4   base_salary         71247 non-null  float64
 5   overtime            38998 non-null  float64
 6   health              52959 non-null  float64
 7   retired             7404 non-null   object 
 8   year                71946 non-null  int64  
 9   city_id             71946 non-null  int64  
 10  irregular_cash      71946 non-null  float64
 11  retirement          71946 non-null  float64
 12  other_benefits      71946 non-null  float64
 13  total_benefits      71946 non-null  float64
 14  total_compensation  71946 non-null  float64
dtypes: float64(9), int64(2), object(4)
memory usage: 8.2+